<a href="https://colab.research.google.com/github/Minseo-202103621/machinelearning_deeplearning/blob/main/CH05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치
## 검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 배운다. 그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터를 찾는다.


이전까지는 테스트 세트에서 모델을 평가해왔다. 그런데 테스트 세트를 사용해 자꾸 성능을 확인하다보면 점점 테스트 세트에 맞추게 된다. 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다. 모델을 만들고 나서 마지막에 한 번만 사용하는 것이 좋다.

* 그렇다면 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝을 어떻게 할 수 있을까? 게다가 결정 트리는 테스트해 볼 매개변수가 많다.

--- 
# 검증 세트
테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소인지 판단하기 어렵다. 그래서 최대한 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는거다. 이 데이터를 검증 세트라고 부른다.

이전에는 80% 훈련 세트 20% 테스트 세트였는데 여기서는 20%를 훈련세트에서 떼어 검증 세트로 만든다.

1.훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다. 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다. 2.이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다. 그리고 마지막에 3.테스트 세트에서 최종 점수를 평가한다.



In [31]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')


In [32]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()


In [33]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state = 42)


* 이제 train_input, train_target을 다시 train_test_split() 함수에 넣어 훈련 세트 sub_input,sub_target과 검증 세트 val_input,val_target을 만든다.

In [34]:
sub_input,val_input,sub_target,val_target = train_test_split(train_input,train_target,test_size=0.2,random_state = 42)
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


* 원래 5,197개였던 훈련 세트가 4,157로 줄고 1,040개의 검증 세트가 완성됐다. 이제 sub_input,sub_target,val_input,val_target을 사용해 모델을 만들고 평가해보겠다.

In [35]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


* 훈련 세트에 과대적합되어 있다. 매개변수를 바꿔서 더 좋은 모델을 찾아야 한다. 
* 그 전에 검증 세트에 관해 좀 더 알아야 할 것이 있다.

---
# 교차 검증
 많은 훈련 데이터가 있어야 좋은 모델이 만들어지는데,,,검증 세트를 만드느라 훈련 세트가 줄었다. 그렇다고 검증을 줄이자니 검증 점수가 불안정할 것이고..

 이럴 때! ***교차검증***을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다. 

k-폴드 교차 검증


이러한 방법을 사용하면 데이터의 80~90%까지 훈련에 사용할 수 있다. 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.

* 사이킷런에서는 cross_validation()라는 교차 검증 함수가 있다. 
* 사용법은 평가할 모델 객체를 첫 번째 매개변수로 전달하고 그 다음 앞에서처럼 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validation()함수에 전달한다.
 



In [36]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.00934172, 0.00812006, 0.00738955, 0.0072701 , 0.00696278]), 'score_time': array([0.00099778, 0.00101471, 0.00089073, 0.00104094, 0.00092936]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


* fit_time,score_time,test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미하고 각 키마다 5개의 숫자가 담겨있다. cross_validate()함수는 기본적으로 5-폴드 교차 검증을 수행한다. cv 매개변수에서 폴드 수를 바꿀 수도 있다.
* 교차 검증의 최종 점수는 test_score 키에 담기 5개의 점수를 평균하여 얻을 수 있다. 이름은 test_score지만 검증 폴드의 점수이다. 

In [37]:
import numpy as np
print(np.mean(scores['test_score']))


0.855300214703487


* cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 한다.
* cross_validate()함수는 기본적으로 회귀 모델일 경우 KFOLD 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [38]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input,train_target,cv = StratifiedKFold())
print(np.mean(scores['test_score']))


0.855300214703487


* 만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성한다.

In [39]:
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt,train_input,train_target,cv =splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


* 이제 이어서 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아 보겠다.

---
# 하이퍼파라미터 튜닝

모델 파라미터 : 머신러닝 모델이 학습하는 파라미터

하이퍼 파라미터 : 모델이 학습할 수 없어서 사용자가 지정해주는 파라미터

사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼 파라미터는 모두 클래스나 메서드의 매개변수로 표현한다.

그럼 어떻게 하이퍼파라미터를 튜닝하는 작업을 하나?>>먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련 >> 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔본다.

그런데 하나의 매개변수의 최적의 값을 찾았고 이 값을 고정하고 다른 매개변수의 최적을 값을 찾아봐야 아무 소용이 없다. 다른 매개 변수를 변경하면 최적의 값을 찾았던  매개변수의 값 또한 변하게 된다. 즉 매개변수들을 동시에 바꿔가며 최적의 값을 찾아야 한다.

* 사이킷런에서 제공하는 그리드 서치를 사용하면 된다.
* 시이킷런의 GridSearchCV 클래스는 친절하게도 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 별도로 cross_validation()함수를 호출할 필요가 없다.
* ex) 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decreas 매개변수의 최적값을 찾아보자. 먼저 GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만들자.



In [40]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001,0.0002,0.0003,0.0004,0.0005]}


* 여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도하겠다. GridSearchCV클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다.


In [41]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)


* GridSearchCV의 cv 매개변수 기본값은 5이다. 따라서 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행한다. 결국 5X5 = 25개의 모델을 훈련한다. 
* 많은 모델을 훈려하기 때문에 n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정하는 것이다 좋다. 기본값은 1이고 -1을 설정해서 모든 코어를 사용한다.


In [42]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

* 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.
* 이 모델은 gs객체의 best_estimator_속성에 저장되어 있다. 일반 결정 트리처럼 똑같이 사용할 수 있다.

In [43]:
dt = gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [44]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


* 각 매개변수에 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장되어 있다.

In [45]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


* 수동으로 고르는 것보다 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다. 그 다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다. 이 값이 최상의 검증 점수를 만든 매개변수 조합이다.

In [46]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


이 과정을 정리해보면
* 먼저 탐색할 매개변수를 지정한다.
* 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.
* 그리드 서치는 최상의 매개변수에서(교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

만약 조금 더 복잡한 매개변수 조합을 탐색할려면 ?? 결정 트리에서 min_impurity_decrease는 노드를 분할 하기 위한 불순도 감소 최소량을 지정한다.
여기에다가 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라 보겠다.

In [47]:
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)}
          

* 이 매개변수로 수행할 교차 검증 횟수는 9X15X10 = 1,350개이다. 기본 5-폴드 교차검증을 수행하므로 만들어지는 모델의 수는 6,570개나 된다. n_jobs 매개변수를 -1로 설정하고 그리드 서치를 실행해 보겠다.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

In [ ]:
print(gs.best_params_)

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

* GridSearchCV 클래스를 사용하니 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개변수를 찾을 수 있다.
* 그런데 앞에서 탐색할 매개변수의 간격을 0.0001 혹은1로 설정했는데 이렇게 간격을 둔 것에 특별한 근거가 없다. 이보다 더 좁거나 넓은 간격으로 시도해 볼 수 있지 않을까?

--- 
# 랜덤 서치
매개변수 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다. 이럴 때 ***랜덤 서치***를 사용하면 좋다.

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다. 

* 먼저 싸이파이에서 2개의 확률 분포 클래스를 임포트해 보겠다.


In [ ]:
from scipy.stats import uniform,randint


* 싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. 이를 균등 분포에서 샘플링한다라고 말한다.
* randint는 정숫값을 뽑고,uniform은 실숫값을 뽑는다. 

In [ ]:
rgen = randint(0,10)
rgen.rvs(10)

In [ ]:
np.unique(rgen.rvs(1000),return_counts=True)

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

In [ ]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}


* 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [58]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f54fdfd9d20>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f54fdfd8e50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f54fdfd91e0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f54fdfd9cc0>},
                   random_state=42)

* 위 params에 정의된 매개변수 범위에서 총 100번(n_iter 매개변수)을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다.
* 앞서 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다.

In [59]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [60]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [61]:
dt = gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
